In [1]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import warnings

# Bỏ qua warning
warnings.filterwarnings('ignore')

# Đọc dữ liệu từ file csv
dataset = pd.read_csv('C:/Users/green/Downloads/DemoDA3/creditcard.csv')

In [2]:
# Chuẩn hóa dữ liệu
dataset['new_amount'] = StandardScaler().fit_transform(dataset['Amount'].values.reshape(-1, 1))
dataset.drop(['Time', 'Amount'], axis=1, inplace=True)
dataset.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,new_amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403


In [3]:
# Chọn 50% dữ liệu giao dịch bất thường để train model 
train_set_percentage = 0.5
fraud_series = dataset[dataset['Class'] == 1]
idx = fraud_series.index.values
np.random.shuffle(idx)
fraud_series.drop(idx[:int(idx.shape[0]*train_set_percentage)], inplace=True)
dataset.drop(fraud_series.index.values, inplace=True)

In [4]:
# Chọn tập dữ liệu bình thường có cùng kích thước với tập giao dịch bất thường
normal_series = dataset[dataset['Class'] == 0] 
idx = normal_series.index.values
np.random.shuffle(idx)
normal_series.drop(idx[fraud_series.shape[0]:], inplace=True)
dataset.drop(normal_series.index.values, inplace=True)

In [5]:
# Gộp dữ liệu thành một tập dữ liệu mới
new_dataset = pd.concat([normal_series, fraud_series])
new_dataset.reset_index(inplace=True, drop=True)
y = new_dataset['Class'].values.reshape(-1, 1)
new_dataset.drop(['Class'], axis=1, inplace=True)
X = new_dataset

In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Các thuộc tính cần thiết
attr={'C': [0.1, 1, 2, 5, 10, 25, 50, 100],
      'gamma': [1e-1, 1e-2, 1e-3]
     }

# Phân tách dataset
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size=0.2, random_state=10)

# Training model
model = SVC()
classif = GridSearchCV(model, attr, cv=5)
classif.fit(X_train, y_train)
y_pred = classif.predict(X_test)
print('Accuracy: ',accuracy_score(y_pred, y_test))

Accuracy:  0.9696969696969697


In [7]:
# Dự đoán cho tập dữ liệu ban đầu
y_all = dataset['Class'].values.reshape(-1, 1)
dataset.drop(['Class'], axis=1, inplace=True)
X_all = dataset
y_pred_all = classif.predict(X_all)

In [8]:
# Kết quả dự đoán
result = confusion_matrix(y_all, y_pred_all)
print("Confusion Matrix:")
print(result)
result1 = classification_report(y_all, y_pred_all)
print("Classification Report:",)
print (result1)
result2 = accuracy_score(y_all, y_pred_all)
print("Accuracy:",result2)

Confusion Matrix:
[[246030  38039]
 [    16    230]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.87      0.93    284069
           1       0.01      0.93      0.01       246

    accuracy                           0.87    284315
   macro avg       0.50      0.90      0.47    284315
weighted avg       1.00      0.87      0.93    284315

Accuracy: 0.8661519793187133
